In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import keras_tuner as kt

from sklearn.model_selection import train_test_split

import pandas as pd
data=pd.read_csv('train.csv')
x=data.iloc[:,3:11]
y=data.iloc[:,-1]

scaler=StandardScaler()


x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=43)

x_train_normalized=scaler.fit_transform(x_train)
x_test_normalized=scaler.fit_transform(x_test)

model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])


model.summary()

model.compile(optimizer='adam',loss='mean_squared_error')

history=model.fit(x_train_normalized,
                 y_train,epochs=200,validation_data=(x_test_normalized,y_test),batch_size=32,verbose=1)

loss=model.evaluate(x_test_normalized,y_test)

y_pred=model.predict(x_test_normalized)

plt.scatter(y_test,y_pred)

plt.plot(history.history['loss'],label='train loss')
plt.plot(history.history['val_loss'],label='val loss')
plt.legend()
plt.show()

def build_model(hp):
    model=Sequential([
        Dense(hp.Int('units1',min_value=32,max_value=128,step=32)),
        Dense(hp.Int('units2',min_value=16,max_value=64,step=16)),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate',min_value=1e-4,max_value=1e-2,sampling='LOG')),
             loss='mean_squared_error')
    return model

tuner=kt.Hyperband(build_model,
               objective='val_loss',
               max_epochs=50,
               hyperband_iterations=2,
               directory='kt_dir',
               project_name='car price')
tuner.search(x_train_normalized,y_train,epochs=50,validation_split=0.2)
best_model=tuner.get_best_models(num_models=1)[0]